# Pull Request n.1 (#5163)

- ## Cosa e' una Transform in PennyLane?

*A quantum transform is a crucial concept in PennyLane, and refers to mapping a quantum circuit to one or more circuits, alongside a classical post-processing function. Once a transform is registered with PennyLane, the transformed circuits will be executed, and the classical post-processing function automatically applied to the outputs. This becomes particularly valuable when a transform generates multiple circuits, requiring a method to aggregate or reduce the results*

Mi sembra che la funzione chiave sia `qml.transform`. *qml.transform can be used to define custom transformations that work with PennyLane QNodes; such transformations can map a circuit to one or many new circuits alongside associated classical post-processing.*
To streamline the creation of transforms and ensure their versatility across various circuit abstractions in PennyLane, `qml.transform` or the `pennylane.transform()` (sono al 100% la stessa cosa) is available. This decorator registers transforms that accept a `QuantumTape` as its primary input, and returns a sequence of QuantumTape **and an associated processing function.**

La TZ scrive che *transforms are composable on the qnode, but they are unwieldy to compose when working with tapes and batches of tapes.* Penso si riferisca al fatto che comporre usando (ad esempio) @qml.compile sopra @qml.node e' in effetti abbastanza semplice.

*PennyLane offers multiple tools for compiling circuits. We use the term “compilation” here in a loose sense as the process of transforming one circuit into one or more differing circuits.* 

**A circuit could be either a quantum function or a sequence of operators**. *For example, such a transformation could replace a gate type with another, fuse gates, exploit mathematical relations that simplify an observable, or replace a large circuit by a number of smaller circuits.*


***Esempio di applicatione di `qml.transform`***

We first define an input quantum transform with the necessary structure defined in https://docs.pennylane.ai/en/stable/code/api/pennylane.transform.html. 

In this example we copy the tape and sum the results of the execution of the two tapes. La somma dei circuiti viene eseguita dalla post-processing function interna. 

NB: come spiegato nella risposta ad una domanda piu' in basso, **un tape e' una particolare data structure di pennylane che puo' rappresentare quantum circuits e measurement statistics.**

In [1]:
import pennylane as qml

from typing import Sequence, Callable


def my_quantum_transform(tape: qml.tape.QuantumTape) -> (Sequence[qml.tape.QuantumTape], Callable):  # type: ignore

    print(f"\n`my_quantum_transform` has been called\n")

    tape1 = tape
    tape2 = tape.copy()

    def post_processing_fn(results):
        print(
            f"The inner post-processing function in `my_quantum_transform` has been called, and results={results}"
        )
        return qml.math.sum(results)

    return [tape1, tape2], post_processing_fn

FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata


We want to be able to apply this transform on both a `qfunc` and a `pennylane.QNode` and will use transform to achieve this. 

**transform validates the signature of your input quantum transform and makes it capable of transforming qfunc and pennylane.QNode in addition to quantum tapes.**

Cioe' in pratica mi sembra che transform serva per passare concretamente dalla trasformazione teorica del circuito che voglio effettuare alla trasformazione pratica, nel senso di agire concretamente su un qnode.
Questo e' confermato anche dalla frase esterna che *in order to make your transform applicable to both QNode and quantum functions, you can use the pennylane.transform() decorator.*

Let’s define a circuit as a pennylane.QNode:

In [2]:
dev = qml.device("default.qubit")


@qml.qnode(device=dev)
def qnode_circuit(a):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.PauliX(wires=0)
    qml.RZ(a, wires=1)
    return qml.expval(qml.PauliZ(wires=0))


print(type(qnode_circuit), "\n")

print(qml.draw(qnode_circuit)(0.2))

<class 'pennylane.workflow.qnode.QNode'> 

0: ──H─╭●──X────────┤  <Z>
1: ────╰X──RZ(0.20)─┤     


We first apply transform to my_quantum_transform:

In [3]:
dispatched_transform = qml.transform(quantum_transform=my_quantum_transform)

print(f"qml.transform returns a TransformDispatcher: {type(dispatched_transform)}")

FRISUS LOG: transform e' stata chiamata
qml.transform returns a TransformDispatcher: <class 'pennylane.transforms.core.transform_dispatcher.TransformDispatcher'>


Now I can use the dispatched transform directly on a `pennylane.QNode`, perche' un TransformDispatcher consente proprio di agire (anche) su un `pennylane.QNode`.

For pennylane.QNode, the dispatched transform populates the TransformProgram of your QNode. The transform and its processing function are applied in the execution.

~~Assumo che qui 'processing function' si riferisca alla funzione che io ho scritto, cioe' my_quantum_transform.~~
Nope, la postprocessing function is a classical function returning a physical information acting on the results of circuit's execution.

In [4]:
transformed_qnode = dispatched_transform(qnode_circuit)

print(f"The TransformDispatcher has returned a QNode: {type(transformed_qnode)}")

print(qml.draw(transformed_qnode)(0.4))

The TransformDispatcher has returned a QNode: <class 'pennylane.workflow.qnode.QNode'>

`my_quantum_transform` has been called

0: ──H─╭●──X────────┤  <Z>
1: ────╰X──RZ(0.40)─┤     

0: ──H─╭●──X────────┤  <Z>
1: ────╰X──RZ(0.40)─┤     


Per chiamare la funzione interna definita dentro a my_quantum_transform, apparentemente devo fare come segue.

Questo non risulta altro che la conferma della frase sopra: **The transform and its processing function are applied in the execution.** Ed io non sto facendo altro che eseguire il circuito.

In [5]:
batch = transformed_qnode(0.4)


`my_quantum_transform` has been called

The inner post-processing function in `my_quantum_transform` has been called, and results=(tensor(0., requires_grad=True), tensor(0., requires_grad=True))


In [6]:
transformed_qnode.transform_program

print(qml.draw(transformed_qnode)(0.4))


`my_quantum_transform` has been called

0: ──H─╭●──X────────┤  <Z>
1: ────╰X──RZ(0.40)─┤     

0: ──H─╭●──X────────┤  <Z>
1: ────╰X──RZ(0.40)─┤     


If we apply dispatched_transform a second time to the pennylane.QNode, we would add it to the transform program again and therefore the transform would be applied twice before execution.

In [7]:
transformed_qnode = dispatched_transform(transformed_qnode)
transformed_qnode.transform_program

print(qml.draw(transformed_qnode)(0.4))


`my_quantum_transform` has been called


`my_quantum_transform` has been called


`my_quantum_transform` has been called

0: ──H─╭●──X────────┤  <Z>
1: ────╰X──RZ(0.40)─┤     

0: ──H─╭●──X────────┤  <Z>
1: ────╰X──RZ(0.40)─┤     

0: ──H─╭●──X────────┤  <Z>
1: ────╰X──RZ(0.40)─┤     

0: ──H─╭●──X────────┤  <Z>
1: ────╰X──RZ(0.40)─┤     


When a transformed QNode is executed, the QNode’s transform program is applied to the generated tape and creates a sequence of tapes to be executed. The execution results are then post-processed in the reverse order of the transform program to obtain the final results. Figo!

In [8]:
transformed_qnode(0.4)


`my_quantum_transform` has been called


`my_quantum_transform` has been called


`my_quantum_transform` has been called

The inner post-processing function in `my_quantum_transform` has been called, and results=(tensor(0., requires_grad=True), tensor(0., requires_grad=True))
The inner post-processing function in `my_quantum_transform` has been called, and results=(tensor(0., requires_grad=True), tensor(0., requires_grad=True))
The inner post-processing function in `my_quantum_transform` has been called, and results=(0.0, 0.0)


0.0

- ## In che senso le transforms sono composable?

*Transforms are inherently composable on a QNode, meaning that transforms with compatible post-processing functions can be successively applied to QNodes.* 

For example, this allows for the application of multiple compilation passes on a QNode to maximize gate reduction before execution:

~~TODO: la documentazione a https://docs.pennylane.ai/en/stable/code/qml_transforms.html deve essere aggiornata, in quanto si utilizzano ancora le transforms come decoratori invece di usare @partial (come richiesto ora). Qui di seguito riporto la documentazione corretta.~~

In [9]:
from pennylane.transforms.optimization import cancel_inverses, merge_rotations

from functools import partial

dev = qml.device("default.qubit", wires=1)


@partial(merge_rotations)
@partial(cancel_inverses)
@qml.qnode(
    device=dev
)  # TODO ci sono due punti di troppo nella documentazione a https://docs.pennylane.ai/en/stable/code/qml_transforms.html in fondo che posso e voglio correggere
def circuit(x, y):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=0)
    qml.RX(x, wires=0)
    qml.RY(y, wires=0)
    qml.RZ(y, wires=0)
    qml.RY(x, wires=0)
    return qml.expval(qml.PauliZ(wires=0))

In [10]:
from typing import Sequence, Callable
from pennylane.tape import QuantumTape


@qml.transform
def sum_circuit_and_adjoint(tape: QuantumTape) -> (Sequence[QuantumTape], Callable):

    operations = [qml.adjoint(op) for op in tape.operation]
    new_tape = type(tape)(operations, tape.measurements, shots=tape.shots)

    def null_postprocessing(results):
        return qml.sum(results)

    return [tape, shifted_tape], null_postprocessing


dev = qml.device("default.qubit", wires=1)


@qml.transforms.merge_rotations
@qml.transforms.cancel_inverses
@qml.qnode(device=dev)
def circuit(x, y):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=0)
    qml.RX(x, wires=0)
    qml.RY(y, wires=0)
    qml.RZ(y, wires=0)
    qml.RY(x, wires=0)
    return qml.expval(qml.PauliZ(wires=0))

FRISUS LOG: transform e' stata chiamata


- ## Che minchia e' un transform dispatcher?

Tecnicamente parlando, `TransformDispatcher` e' un tipo, cioe' una classe, returnato da qml.transform. 

Stando a https://docs.pennylane.ai/en/stable/code/api/pennylane.transforms.core.transform_dispatcher.TransformDispatcher.html, *converts a transform that has the signature (tape -> Sequence(tape), fn) to a transform dispatcher that can act on pennylane.tape.QuantumTape, quantum function, pennylane.QNode, pennylane.devices.Device.*



- ## Passando ad un esempio concreto e reale di transform, che cosa fa nel dettaglio compile()?

***Compilation functionality is mostly designed as transforms***

The default behaviour of compile() applies a sequence of three transforms: commute_controlled(), cancel_inverses(), and then merge_rotations().

WITHOUT @qml.compile

In [11]:
dev = qml.device("default.qubit", wires=[0, 1, 2])


@qml.qnode(dev)
def circuitNC(x, y, z):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    qml.RZ(z, wires=2)
    qml.CNOT(wires=[2, 1])
    qml.RX(z, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RX(x, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RZ(-z, wires=2)
    qml.RX(y, wires=2)
    qml.PauliY(wires=2)
    qml.CZ(wires=[1, 2])
    return qml.expval(qml.PauliZ(wires=0))


print(qml.draw(circuitNC)(0.2, 0.3, 0.4))

0: ──H──RX(0.40)────╭X──────────RX(0.20)─╭X────┤  <Z>
1: ──H───────────╭X─╰●───────────────────╰●─╭●─┤     
2: ──H──RZ(0.40)─╰●──RZ(-0.40)──RX(0.30)──Y─╰Z─┤     


WITH @qml.compile

In [12]:
dev = qml.device("default.qubit", wires=[0, 1, 2])


@qml.compile
@qml.qnode(dev)
def circuit(x, y, z):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    qml.RZ(z, wires=2)
    qml.CNOT(wires=[2, 1])
    qml.RX(z, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RX(x, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RZ(-z, wires=2)
    qml.RX(y, wires=2)
    qml.PauliY(wires=2)
    qml.CZ(wires=[1, 2])
    return qml.expval(qml.PauliZ(wires=0))


print(qml.draw(circuit)(0.2, 0.3, 0.4))

FRISUS LOG: compile e' stata chiamata
0: ──H──RX(0.60)─────────────────┤  <Z>
1: ──H─╭X─────────────────────╭●─┤     
2: ──H─╰●─────────RX(0.30)──Y─╰Z─┤     


Partendo dal primo circuito non compilato circuito NC, posso compilare direttamente anche cosi':

In [13]:
compiled_circuit = qml.compile(circuitNC)
compiled_qnode = qml.QNode(compiled_circuit, dev)
print(qml.draw(compiled_circuit)(0.2, 0.3, 0.4))
# print(qml.draw(compiled_qnode)(0.2, 0.3, 0.4)) NB questo printa il circuito NON compilato, nonostante sia presente nella documentazione.
# Che sia un "bug" nella documentazione? Spero di si, cosi' ho qualcosa da proporre!

FRISUS LOG: compile e' stata chiamata
0: ──H──RX(0.60)─────────────────┤  <Z>
1: ──H─╭X─────────────────────╭●─┤     
2: ──H─╰●─────────RX(0.30)──Y─╰Z─┤     


Cioe' in pratica compile() ha trasformato un ciclo in un altro. ~~Immagino che abbia preservato una qualche proprieta' del circuito stesso.~~ (si, lo ha solo semplificato ma il risultato non risulta cambiato). 

Da quei FDP vedo che compile() chiama 3 sotto-trasformazioni. E se io volessi chiamarne solo una, ad esempio commute_controlled()? Mi sembra talmente banale che non vale neanche la pena di farlo (comunque l'ho fatto sotto)

In [14]:
from pennylane.transforms.optimization import (
    cancel_inverses,
    commute_controlled,
    merge_rotations,
    remove_barrier,
)

compile_default_pipeline = [commute_controlled, cancel_inverses, merge_rotations, remove_barrier]

dev = qml.device("default.qubit", wires=[0, 1, 2])


@qml.qnode(dev)
def circuit(x, y, z):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    qml.RZ(z, wires=2)
    qml.CNOT(wires=[2, 1])
    qml.RX(z, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RX(x, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RZ(-z, wires=2)
    qml.RX(y, wires=2)
    qml.PauliY(wires=2)
    qml.CZ(wires=[1, 2])
    return qml.expval(qml.PauliZ(wires=0))


print(qml.draw(circuit)(0.2, 0.3, 0.4))

transformed_circuit = qml.transforms.commute_controlled(circuit)

0: ──H──RX(0.40)────╭X──────────RX(0.20)─╭X────┤  <Z>
1: ──H───────────╭X─╰●───────────────────╰●─╭●─┤     
2: ──H──RZ(0.40)─╰●──RZ(-0.40)──RX(0.30)──Y─╰Z─┤     


Per usare solo una trasform di `compile`, posso fare come segue (copiato dalla documentation di PennyLane):

In [15]:
from pennylane.transforms.optimization import (
    cancel_inverses,
    commute_controlled,
    merge_rotations,
    remove_barrier,
)

from functools import partial

compile_default_pipeline = [commute_controlled, cancel_inverses, merge_rotations, remove_barrier]

dev = qml.device("default.qubit", wires=[0, 1, 2])


@partial(qml.compile, pipeline=[commute_controlled])
# OR @partial(commute_controlled) instead of the one above, which produces exactly the same output
@qml.qnode(dev)
def circuit(x, y, z):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.Hadamard(wires=2)
    qml.RZ(z, wires=2)
    qml.CNOT(wires=[2, 1])
    qml.RX(z, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RX(x, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RZ(-z, wires=2)
    qml.RX(y, wires=2)
    qml.PauliY(wires=2)
    qml.CZ(wires=[1, 2])
    return qml.expval(qml.PauliZ(wires=0))


print(qml.draw(circuit)(0.2, 0.3, 0.4))

FRISUS LOG: compile e' stata chiamata
0: ──H────╭X────────╭X──────────RX(0.40)──RX(0.20)────┤  <Z>
1: ──H─╭X─╰●────────╰●─────────────────────────────╭●─┤     
2: ──H─╰●──RZ(0.40)──RZ(-0.40)──RX(0.30)──Y────────╰Z─┤     



- ## Che cazzo e' una tape in PennyLane?

Dalla description di qml.compile, mi pare di intuire che un *tape* sia solo un modo stronzo di dire circuito.

Piu' nel dettaglio, stando a https://docs.pennylane.ai/en/stable/code/qml_tape.html, *Quantum tapes are a datastructure that can represent quantum circuits and measurement statistics in PennyLane. They are queuing contexts that can record and process quantum operations and measurements. In addition to being created internally by QNodes, quantum tapes can also be created, nested, expanded (via expand()), and executed manually.*

- ## Che cazzo e' una batch in PennyLane?

In the pennylane source code, a "batch" is a fundamental pennylane object.

- ## Cosa fa esattamente `map_batch_transform` e come posso usarla? 

**Questo turns out to be estremamente rilevante per quello che devo fare in questa PR.**

Per capire sta roba, supponiamo di avere due `tapes` seguenti:

In [36]:
from pennylane.transforms import (
    map_batch_transform,
)  # Questa linea consente di vedere qualcosa hooverando con vscode

from pennylane.transforms import (
    hamiltonian_expand,
)  # Questa linea consente di vedere qualcosa hooverando con vscode

H = qml.PauliZ(0) @ qml.PauliZ(1) - qml.PauliX(0)

ops1 = [qml.RX(0.5, wires=0), qml.RY(0.1, wires=1), qml.CNOT(wires=(0, 1))]
measurements1 = [qml.expval(H)]
tape1 = qml.tape.QuantumTape(ops1, measurements1)

ops2 = [qml.Hadamard(0), qml.CRX(0.5, wires=(0, 1)), qml.CNOT((0, 1))]
measurements2 = [qml.expval(H + 0.5 * qml.PauliY(0))]
tape2 = qml.tape.QuantumTape(ops2, measurements2)

Noto che questi tapes non sono Qnodes, ma c'e' comunque informazione sul circuito.

We can use `map_batch_transform` to map a single transform across both of the these tapes in such a way that allows us to submit a single job for execution:

In [39]:
tapes, fn = map_batch_transform(transform=hamiltonian_expand, tapes=[tape1, tape2])

dev = qml.device("default.qubit", wires=2)

fn(qml.execute(tapes, dev, qml.gradients.param_shift))


Executing `map_batch_transform` MODIFIED BY FRISUS

input transform=<transform: hamiltonian_expand>
input tapes=[<QuantumTape: wires=[0, 1], params=4>, <QuantumTape: wires=[0, 1], params=4>]


current tape in for cycle=<QuantumTape: wires=[0, 1], params=4>
FRISUS LOG: hamiltonian_expand e' stata chiamata
new_tapes=[<QuantumTape: wires=[0, 1], params=2>, <QuantumTape: wires=[0, 1], params=2>]
fn=<function hamiltonian_expand.<locals>.processing_fn at 0x7fb4650ab8b0>


current tape in for cycle=<QuantumTape: wires=[0, 1], params=4>
FRISUS LOG: hamiltonian_expand e' stata chiamata
new_tapes=[<QuantumTape: wires=[0, 1], params=1>, <QuantumTape: wires=[0, 1], params=1>, <QuantumTape: wires=[0, 1], params=1>]
fn=<function hamiltonian_expand.<locals>.processing_fn at 0x7fb464912160>

map_batch_transform.processing_fn inner function has been called.
FRISUS LOG: e' stata chiamata la postp. function within hamiltonian_expand
Sta agendo su res=(0.0, 0.9950041652780257)
FRISUS LOG: e' stata chiama

[array(0.99500417), array(0.8150893)]

In [18]:
map_batch_transform(transform=hamiltonian_expand, tapes=[tape1, tape2])


Executing `map_batch_transform` MODIFIED BY FRISUS

input transform=<transform: hamiltonian_expand>
input tapes=[<QuantumTape: wires=[0, 1], params=4>, <QuantumTape: wires=[0, 1], params=4>]


current tape in for cycle=<QuantumTape: wires=[0, 1], params=4>
FRISUS LOG: hamiltonian_expand e' stata chiamata
new_tapes=[<QuantumTape: wires=[0, 1], params=2>, <QuantumTape: wires=[0, 1], params=2>]
fn=<function hamiltonian_expand.<locals>.processing_fn at 0x7fb464fcfd30>


current tape in for cycle=<QuantumTape: wires=[0, 1], params=4>
FRISUS LOG: hamiltonian_expand e' stata chiamata
new_tapes=[<QuantumTape: wires=[0, 1], params=1>, <QuantumTape: wires=[0, 1], params=1>, <QuantumTape: wires=[0, 1], params=1>]
fn=<function hamiltonian_expand.<locals>.processing_fn at 0x7fb464fcfe50>



([<QuantumTape: wires=[0, 1], params=2>,
  <QuantumTape: wires=[0, 1], params=2>,
  <QuantumTape: wires=[0, 1], params=1>,
  <QuantumTape: wires=[0, 1], params=1>,
  <QuantumTape: wires=[0, 1], params=1>],
 <function pennylane.transforms.batch_transform.map_batch_transform.<locals>.processing_fn(res: Tuple[~Result]) -> Tuple[~Result]>)

- ## Che roba e' un `QuantumScript`?

Un QuantumScript e' un tipo (cioe' una classe) che rappresenta le istruzioni da eseguire su un quantum device

Da https://docs.pennylane.ai/en/stable/code/api/pennylane.tape.QuantumScript.html copio il seguente snippettino che fa vedere cosa sia

In [40]:
from pennylane.tape import QuantumScript

from pennylane import numpy as np

ops = [
    qml.BasisState(np.array([1, 1]), wires=(0, "a")),
    qml.RX(0.432, 0),
    qml.RY(0.543, 0),
    qml.CNOT((0, "a")),
    qml.RX(0.133, "a"),
]

qscript = QuantumScript(ops=ops, measurements=[qml.expval(qml.PauliZ(0))])

Da qui apprezziamo che qscript contiene sia operations che measurements (unione di due insiemi disgiunti)

In [20]:
list(qscript)

[BasisState(tensor([1, 1], requires_grad=True), wires=[0, 'a']),
 RX(0.432, wires=[0]),
 RY(0.543, wires=[0]),
 CNOT(wires=[0, 'a']),
 RX(0.133, wires=['a']),
 expval(Z(0))]

In [21]:
qscript.operations

[BasisState(tensor([1, 1], requires_grad=True), wires=[0, 'a']),
 RX(0.432, wires=[0]),
 RY(0.543, wires=[0]),
 CNOT(wires=[0, 'a']),
 RX(0.133, wires=['a'])]

In [22]:
qscript.measurements

[expval(Z(0))]

- ## Qual e' l'opzione attuale per comporre due transforms quando si lavora con il tape paradigm?

**NB Questo e' riportato anche nella descizione della mia storia**

In [41]:
H = qml.PauliZ(0) @ qml.PauliZ(1) - qml.PauliX(0)

ops1 = [qml.RX(0.5, wires=0), qml.RY(0.1, wires=1), qml.CNOT(wires=(0, 1))]
measurements1 = [qml.expval(H)]
tape1 = qml.tape.QuantumTape(ops1, measurements1)

print(f"tape1={tape1}")

ops2 = [qml.Hadamard(0), qml.CRX(0.5, wires=(0, 1)), qml.CNOT((0, 1))]
measurements2 = [qml.expval(H + 0.5 * qml.PauliY(0))]
tape2 = qml.tape.QuantumTape(ops2, measurements2)

print(f"tape2={tape2}")

tape1=<QuantumTape: wires=[0, 1], params=4>
tape2=<QuantumTape: wires=[0, 1], params=4>


In [24]:
# NB se provo a compilare con tape=tape1 mi da errore! TODO indagare...
batch1, fn1 = hamiltonian_expand(tape1)

print(f"batch1={batch1}, \nfn1={fn1}")
# fn1 e' la processing function interna definita dentro hamiltonian expand.

batch2, fn2 = map_batch_transform(transform=hamiltonian_expand, tapes=[tape2])


def combined_postprocessing(results):
    return fn1(fn2(results))


combined_postprocessing(
    results=(qml.numpy.tensor(0.0, requires_grad=True), qml.numpy.tensor(0.0, requires_grad=True))
)

FRISUS LOG: hamiltonian_expand e' stata chiamata
batch1=[<QuantumTape: wires=[0, 1], params=2>, <QuantumTape: wires=[0, 1], params=2>], 
fn1=<function hamiltonian_expand.<locals>.processing_fn at 0x7fb46505e550>

Executing `map_batch_transform` MODIFIED BY FRISUS

input transform=<transform: hamiltonian_expand>
input tapes=[<QuantumTape: wires=[0, 1], params=4>]


current tape in for cycle=<QuantumTape: wires=[0, 1], params=4>
FRISUS LOG: hamiltonian_expand e' stata chiamata
new_tapes=[<QuantumTape: wires=[0, 1], params=1>, <QuantumTape: wires=[0, 1], params=1>, <QuantumTape: wires=[0, 1], params=1>]
fn=<function hamiltonian_expand.<locals>.processing_fn at 0x7fb46505e310>

map_batch_transform.processing_fn inner function has been called.
FRISUS LOG: e' stata chiamata la postp. function within hamiltonian_expand
Sta agendo su res=(tensor(0., requires_grad=True), tensor(0., requires_grad=True))
The final results of the post processing functions are {final_results}
FRISUS LOG: e' stata c

tensor(0., requires_grad=True)

- ## Qual e' la soluzione proposta?

In [42]:
from pennylane.transforms import compile, broadcast_expand, transform

qml.drawer.use_style("black_white")

print(f"SET-UP")
print(f"####################################################")
# Device used to test the tape
dev = qml.device("default.qubit", wires=3)
print(f"dev={dev}")
first_transform = hamiltonian_expand
second_transform = broadcast_expand
# Let's create an Hamiltonian and a tape
H = (
    qml.PauliY(wires=2) @ qml.PauliZ(wires=1)
    + 0.5 * qml.PauliZ(wires=2)
    + qml.PauliZ(wires=1)
    + qml.PauliZ(wires=0)
)
ops = [qml.Hadamard(wires=0), qml.CNOT(wires=(0, 1)), qml.PauliX(wires=2)]
measurements = [qml.expval(H)]
tape = QuantumTape(ops, measurements)
# print(f"H={H}")
# print(f"ops={ops}")
# print(f"tape={tape}")
# print(f"measurements={measurements}")
print(f"printed tape:\n", qml.tape.QuantumTape.draw(tape))
print(f"first_transform={first_transform}")
print(f"second_transform={second_transform}")
print(f"####################################################")

print(f"\nEXECUTING FIRST TRANSFORM ON TAPE")
print(f"####################################################")
batch1, fn1 = first_transform(tape)
print(f"batch1={batch1}")
for i in range(len(batch1)):
    print(f"printed tape {i}:\n", qml.tape.QuantumTape.draw(batch1[i]))
print(f"fn1={fn1}")
res = dev.execute(batch1)
print(f"result of execution={res}")
print(f"result of post proc. func={fn1(res)}")
print(f"####################################################")

# TODO: capire perche' questo risulta diverso
print(f"\nEXECUTING (FIRST) TRANSFORM ON TAPE USING transform + dispatched_transform")
print(f"####################################################")
dispatched_transform1 = transform(first_transform)
batch1exp, fn1exp = dispatched_transform1(tape)
print(f"batch1exp={batch1exp}")
for i in range(len(batch1exp)):
    print(f"printed tape {i}:\n", qml.tape.QuantumTape.draw(batch1exp[i]))
print(f"fn1exp={fn1exp}")
res = dev.execute(batch1exp)
print(f"result of execution={res}")
print(f"result of post proc. func={fn1exp(res)}")
print(f"####################################################")
# Anche se ancora non sono sicuro, questo modo esplicito di usare transform
# dovrebbe servire per generalizzare una funzione che trasforma tapes in
# qualcosa che puo trasformare tante altre cose. Quindi forse e per questo
# che batch1 e batch1exp non sono la stessa cosa, anche se i risultati delle esecuzioni sono gli stessi.

print(f"\nEXECUTING SECOND TRANSFORM ON TAPE")
print(f"####################################################")
batch2, fn2 = second_transform(tape)
print(f"batch2={batch2}")
for i in range(len(batch2)):
    print(f"printed tape {i}:\n", qml.tape.QuantumTape.draw(batch2[i]))
print(f"fn2={fn2}")
res = dev.execute(batch2)
print(f"result of execution={res}")
print(f"result of post proc. func={fn2(res)}")
print(f"####################################################")

print(f"\nEXECUTING SECOND TRANSFORM ON BATCH OF TAPES FROM FIRST TRANSFORM")
print(f"####################################################")
batch2, fn2 = second_transform(batch1)
print(f"batch2={batch2}")
print(f"fn2={fn2}")
res = dev.execute(batch2)
print(f"result of execution={res}")
print(f"result of post proc. func={fn2(res)}")
print(f"####################################################")


print(f"\nCOMBINING THE TWO POST-PROCESSING FUNCTIONS")
print(f"####################################################")


def combined_postprocessing(results):
    return fn1(fn2(results))


combined_postprocessing(results=res)
print(f"####################################################")

SET-UP
####################################################
dev=<default.qubit device (wires=3) at 0x7fb46490e880>
printed tape:
 0: ──H─╭●─┤ ╭<𝓗>
1: ────╰X─┤ ├<𝓗>
2: ──X────┤ ╰<𝓗>
first_transform=<transform: hamiltonian_expand>
second_transform=<transform: broadcast_expand>
####################################################

EXECUTING FIRST TRANSFORM ON TAPE
####################################################
FRISUS LOG: hamiltonian_expand e' stata chiamata
batch1=[<QuantumTape: wires=[0, 1, 2], params=0>, <QuantumTape: wires=[0, 1, 2], params=0>]
printed tape 0:
 0: ──H─╭●─┤  <Z>
1: ────╰X─┤  <Z>
2: ──X────┤  <Z>
printed tape 1:
 0: ──H─╭●─┤       
1: ────╰X─┤ ╭<Y@Z>
2: ──X────┤ ╰<Y@Z>
fn1=<function hamiltonian_expand.<locals>.processing_fn at 0x7fb464fcf8b0>
result of execution=((-0.9999999999999998, 0.0, 0.0), 0.0)
FRISUS LOG: e' stata chiamata la postp. function within hamiltonian_expand
Sta agendo su res=((-0.9999999999999998, 0.0, 0.0), 0.0)
result of post proc. func=-0.49999

In [26]:
results = (
    qml.numpy.tensor([0.0, 1.0, 5.0], requires_grad=True),
    qml.numpy.tensor([1.5, 3.0, 6.8], requires_grad=True),
)

fn2(results)

FRISUS LOG: e' stata chiamata la postp. function within _batch_transform
Sta agendo su res=(tensor([0., 1., 5.], requires_grad=True), tensor([1.5, 3. , 6.8], requires_grad=True))


(tensor([0., 1., 5.], requires_grad=True),
 tensor([1.5, 3. , 6.8], requires_grad=True))

**NB** (Scrivo questo dopo che la PR e' stata approvata e il mio livello di conoscenza generale risulta a malapena superiore rispetto a quella che avevo quando scrissi questo notebook in the first place e non uso la punteggiatura STI CAZZI).

Se dichiaro un quantum tape con `QuantumTape`, questo viene recognized as an instance of `QuantumScript`. However, if I declare a tape as `QuantumScript`, this is not recognized as instance of `QuantumTape`. Interessante!

# First review - Required actions

- ### First thing to remember: always check argument name when copying and pasting

- ### Second thing to remember: It is better to loop over objects themselves, rather than loop over an index into them (using `zip`)

- ### Third thing to remember: Tuples are better than lists. In fact, tuples are both more performant and protected from accidental mutation. Basically lists get optimized for the ability to be quickly mutated. This means they take up a lot more space and have an extra level of indirection on lookup.

- ## Adding additional test(s)

Message from LZ:

**I think I'd actually prefer to add an additional test or two that is a bit more concrete and specific. While this technically works to cover the behaviour, there could be a lot of things not being caught. Are the postprocessing functions actually being called on the right slices?**

**Potentially we could add some batch tests to some of the concrete transforms, like `tests/transforms/test_split_non_commuting.py`, `tests/transforms/test_broadcast_expand.py`, and `tests/transforms/test_optimization/test_merge_rotations.py` or something.**

Another message from LZ in a second round of review:

**I'm aiming to be able to replace map_batch_transform and deprecate it. Luckily in this case, we know exactly what the output should be. The first tape should split in two, one measuring [qml.expval(qml.X(0)), qml.expval(qml.X(1))] and the second measuring [qml.expval(qml.Z(0)), qml.expval(qml.Y(0))].**

**We should also test function too.**

**Since the output for `split_non_commuting` is just reshuffling, one thing I like to do in this case is use strings in place of the result values, so we can focus on their reshuffling. Something like:**

``
result = ("a", "b", "c", "d") assert fn(result) == [("a", "b"), ("c", d")]
``

- ## First test I added

In [28]:
from pennylane.transforms import split_non_commuting

"""Test that `split_non_commuting` can transform a batch of tapes"""

# create a batch with two fictitious tapes
tape1 = qml.tape.QuantumScript(
    [qml.RX(1.2, 0)], [qml.expval(qml.X(0)), qml.expval(qml.Y(0)), qml.expval(qml.X(1))]
)
tape2 = qml.tape.QuantumScript([qml.RY(0.5, 0)], [qml.expval(qml.Z(0)), qml.expval(qml.Y(0))])
batch = [tape1, tape2]

# test transform on the batch
new_batch, post_proc_fn = split_non_commuting(batch)

# test that transform has been applied correctly on the batch by explicitly comparing with splitted tapes
tp1 = qml.tape.QuantumScript([qml.RX(1.2, 0)], [qml.expval(qml.X(0)), qml.expval(qml.X(1))])
tp2 = qml.tape.QuantumScript([qml.RX(1.2, 0)], [qml.expval(qml.Y(0))])
tp3 = qml.tape.QuantumScript([qml.RY(0.5, 0)], [qml.expval(qml.Z(0))])
tp4 = qml.tape.QuantumScript([qml.RY(0.5, 0)], [qml.expval(qml.Y(0))])


assert all(qml.equal(tape1, tape2) for tape1, tape2 in zip(new_batch, [tp1, tp2, tp3, tp4]))

# test postprocessing function applied to the transformed batch
assert all(
    qml.equal(tapeA, tapeB)
    for sublist1, sublist2 in zip(post_proc_fn(new_batch), ((tp1, tp2), (tp3, tp4)))
    for tapeA, tapeB in zip(sublist1, sublist2)
)

# final (double) check: test postprocessing function on a fictitious results
result = ("expval1", "expval2", "expval3", "expval4")
assert post_proc_fn(result) == (("expval1", "expval2"), ("expval3", "expval4"))

FRISUS LOG: my new function _batch_transform function e stata chiamata
FRISUS LOG: e' stata chiamata la postp. function within _batch_transform
Sta agendo su res=(<QuantumScript: wires=[0, 1], params=1>, <QuantumScript: wires=[0], params=1>, <QuantumScript: wires=[0], params=1>, <QuantumScript: wires=[0], params=1>)
FRISUS LOG: e' stata chiamata la postp. function within _batch_transform
Sta agendo su res=('expval1', 'expval2', 'expval3', 'expval4')


- ## Second test I added

I want to be sure that post-processing functions are called the same way as before

In [29]:
from pennylane.transforms import compile, broadcast_expand, transform

from pennylane.typing import TensorLike


def first_valid_transform(
    tape: qml.tape.QuantumTape, index: int
) -> (Sequence[qml.tape.QuantumTape], Callable): # type: ignore
    """A valid transform."""
    tape = tape.copy()
    tape._ops.pop(index)  # pylint:disable=protected-access
    _ = (qml.PauliX(0), qml.S(0))
    return [tape], lambda x: x


def second_valid_transform(
    tape: qml.tape.QuantumTape, index: int
) -> (Sequence[qml.tape.QuantumTape], Callable): # type: ignore
    """A valid trasnform."""
    tape1 = tape.copy()
    tape2 = tape.copy()
    tape._ops.pop(index)  # pylint:disable=protected-access

    def fn(results):
        return qml.math.sum(results)

    return [tape1, tape2], fn


valid_transforms = [first_valid_transform, second_valid_transform]

valid_transform = valid_transforms[0]

"""Test that dispatcher can dispatch onto a batch of tapes."""


def check_batch(batch):
    return isinstance(batch, Sequence) and all(
        isinstance(tape, qml.tape.QuantumScript) for tape in batch
    )


def comb_postproc(results: TensorLike, fn1: Callable, fn2: Callable):
    return fn1(fn2(results))


# Create a simple tape

H = qml.PauliY(2) @ qml.PauliZ(1) + 0.5 * qml.PauliZ(2) + qml.PauliZ(1)
measur = [qml.expval(H)]
ops = [qml.Hadamard(0), qml.RX(0.2, 0), qml.RX(0.6, 0), qml.CNOT((0, 1))]

tape = qml.tape.QuantumTape(ops, measur)

############################################################
### Test with two elementary user-defined transforms
############################################################
dispatched_transform1 = transform(valid_transform)
dispatched_transform2 = transform(valid_transform)
batch1, fn1 = dispatched_transform1(tape, index=0)
batch2, fn2 = dispatched_transform2(batch1, index=0)

result = dev.execute(batch2)

assert check_batch(batch1) and check_batch(batch2)
assert isinstance(comb_postproc(result, fn1, fn2), TensorLike)

############################################################
### Test with two `concrete` transforms
############################################################
batch1, fn1 = qml.transforms.hamiltonian_expand(tape)
batch2, fn2 = qml.transforms.merge_rotations(batch1)
result = dev.execute(batch2)


# check that final batch and post-processing functions are what we expect after the two transforms
final_ops = [qml.Hadamard(0), qml.RX(0.8, 0), qml.CNOT([0, 1])]
final_batch = (
    qml.tape.QuantumTape(final_ops, [qml.expval(qml.PauliZ(2)), qml.expval(qml.PauliZ(1))]),
    qml.tape.QuantumTape(final_ops, [qml.expval(qml.PauliY(2) @ qml.PauliZ(1))]),
)
numerical_margin = 1e-8

assert all(qml.equal(tapeA, tapeB) for tapeA, tapeB in zip(final_batch, batch2))
assert abs(comb_postproc(result, fn1, fn2).item() - 0.5) < numerical_margin

FRISUS LOG: transform e' stata chiamata
FRISUS LOG: transform e' stata chiamata
FRISUS LOG: my new function _batch_transform function e stata chiamata
FRISUS LOG: e' stata chiamata la postp. function within _batch_transform
Sta agendo su res=(array(1.32533561),)
FRISUS LOG: hamiltonian_expand e' stata chiamata
FRISUS LOG: my new function _batch_transform function e stata chiamata
FRISUS LOG: e' stata chiamata la postp. function within _batch_transform
Sta agendo su res=((0.9999999999999998, 0.0), 0.0)
FRISUS LOG: e' stata chiamata la postp. function within hamiltonian_expand
Sta agendo su res=((0.9999999999999998, 0.0), 0.0)


In [30]:
print(tape)
print(batch1)
print(batch2)

<QuantumTape: wires=[0, 1, 2], params=5>
[<QuantumTape: wires=[0, 1, 2], params=2>, <QuantumTape: wires=[0, 1, 2], params=2>]
(<QuantumTape: wires=[0, 1, 2], params=1>, <QuantumTape: wires=[0, 1, 2], params=1>)


In [31]:
print(f"printed original tape:\n", qml.tape.QuantumTape.draw(tape))

printed original tape:
 0: ──H──RX──RX─╭●─┤     
1: ────────────╰X─┤ ╭<𝓗>
2: ───────────────┤ ╰<𝓗>


In [32]:
for i in range(len(batch1)):
    print(f"printed intermediate tape {i}:\n", qml.tape.QuantumTape.draw(batch1[i]))

printed intermediate tape 0:
 0: ──H──RX──RX─╭●─┤     
1: ────────────╰X─┤  <Z>
2: ───────────────┤  <Z>
printed intermediate tape 1:
 0: ──H──RX──RX─╭●─┤       
1: ────────────╰X─┤ ╭<Y@Z>
2: ───────────────┤ ╰<Y@Z>


In [33]:
for i in range(len(batch2)):
    print(f"printed final tape {i}:\n", qml.tape.QuantumTape.draw(batch2[i]))

printed final tape 0:
 0: ──H──RX─╭●─┤     
1: ────────╰X─┤  <Z>
2: ───────────┤  <Z>
printed final tape 1:
 0: ──H──RX─╭●─┤       
1: ────────╰X─┤ ╭<Y@Z>
2: ───────────┤ ╰<Y@Z>


In [34]:
for i in range(len(batch1)):
    print(f"printed tape {i}:\n", qml.tape.QuantumTape.draw(batch1[i]))

printed tape 0:
 0: ──H──RX──RX─╭●─┤     
1: ────────────╰X─┤  <Z>
2: ───────────────┤  <Z>
printed tape 1:
 0: ──H──RX──RX─╭●─┤       
1: ────────────╰X─┤ ╭<Y@Z>
2: ───────────────┤ ╰<Y@Z>
